Bu laboratuvar çalışmasında, bu bölümde ele alınan yeniden örnekleme tekniklerini inceliyoruz.
Bu laboratuvardaki bazı komutların bilgisayarınızda çalışması biraz zaman alabilir.
Yine, çoğu import işlemini bu en üst düzeyde yaparak başlıyoruz.

In [1]:
import numpy as np
import statsmodels.api as sm
from ISLP import load_data
from ISLP.models import (ModelSpec as MS,
summarize,
poly)
from sklearn.model_selection import train_test_split

Bu laboratuvar için birkaç yeni import (kütüphane içe aktarma) gereklidir.

In [2]:
from functools import partial
from sklearn.model_selection import \
(cross_validate,
KFold,
ShuffleSplit)
from sklearn.base import clone
from ISLP.models import sklearn_sm

# 5.3.1 Doğrulama Seti Yöntemi

Farklı doğrusal modelleri Auto veri setine uyguladıktan sonra ortaya çıkan test hata oranlarını tahmin etmek için **doğrulama seti yöntemini** inceliyoruz.

Veriyi **eğitim (training)** ve **doğrulama (validation)** setlerine ayırmak için `train_test_split()` fonksiyonunu kullanıyoruz. 392 gözlem olduğundan, `test_size=196` argümanı ile veriyi **eşit büyüklükte iki sete** (her biri 196 gözlem) bölüyoruz.

Böyle rastgelelik içeren işlemler yaparken, sonuçların ileride tam olarak tekrar üretilebilmesi için genellikle bir **rastgelelik tohumunu (random seed)** belirlemek iyi bir fikirdir. Biz de bölücüye `random_state=0` argümanını vererek rastgelelik tohumunu ayarlıyoruz.

In [3]:
Auto = load_data('Auto')
Auto_train, Auto_valid = train_test_split(Auto, test_size=196, random_state=0)

Şimdi, yalnızca **eğitim setine (Auto_train) ait gözlemleri** kullanarak bir doğrusal regresyon modeli uydurabiliriz.

In [4]:
hp_mm = MS(['horsepower'])
X_train = hp_mm.fit_transform(Auto_train)
y_train = Auto_train['mpg']
model = sm.OLS(y_train, X_train)
results = model.fit()

Şimdi, **`predict()`** metodunu kullanarak bu model için oluşturulan model matrisi üzerinde tahminler yapıyoruz; bu tahminler **doğrulama veri seti** kullanılarak hesaplanıyor. Ayrıca, modelimizin **doğrulama ortalama kare hatasını (validation MSE)** da hesaplıyoruz.

In [5]:
X_valid = hp_mm.transform(Auto_valid)
y_valid = Auto_valid['mpg']
valid_pred = results.predict(X_valid)
np.mean((y_valid-valid_pred)**2)

23.61661706966988

Böylece, doğrusal regresyon uyumunun **doğrulama MSE tahmini 23,62** olarak bulunmuştur.

Ayrıca, **daha yüksek dereceli polinom regresyonlar** için de doğrulama hatasını tahmin edebiliriz. Bunun için önce, bir **model stringi**, eğitim ve test setini alan ve test seti üzerinde **MSE** döndüren bir `evalMSE()` fonksiyonu tanımlıyoruz.

In [6]:
def evalMSE(terms, response, train, test):
    mm = MS(terms)
    X_train = mm.fit_transform(train)
    y_train = train[response]
    X_test = mm.transform(test)
    y_test = test[response]
    results = sm.OLS(y_train, X_train).fit()
    test_pred = results.predict(X_test)
    return np.mean((y_test- test_pred)**2)

Hadi bu fonksiyonu kullanarak, doğrusal, ikinci dereceden (kuadratik) ve üçüncü dereceden (kübik) uyumlar için **doğrulama MSE’sini tahmin edelim.**

Burada **`enumerate()`** fonksiyonunu kullanıyoruz; bu fonksiyon, bir **for döngüsü** ile iterasyon yaparken hem **nesnelerin değerlerini** hem de **indekslerini** verir.

In [7]:
MSE = np.zeros(3)
for idx, degree in enumerate(range(1, 4)):
    MSE[idx] = evalMSE([poly('horsepower', degree)],
            'mpg',
            Auto_train,
            Auto_valid)
MSE

array([23.61661707, 18.76303135, 18.79694163])

Bu hata oranları sırasıyla **23,62**, **18,76** ve **18,80**’dir. Farklı bir eğitim/doğrulama bölmesi seçersek, doğrulama setinde biraz farklı hata değerleri bekleyebiliriz.

In [8]:
Auto_train, Auto_valid = train_test_split(Auto,
                                        test_size=196,
                                        random_state=3)
MSE = np.zeros(3)

for idx, degree in enumerate(range(1, 4)):
    MSE[idx] = evalMSE([poly('horsepower', degree)],
    'mpg',
    Auto_train,
    Auto_valid)
MSE

array([20.75540796, 16.94510676, 16.97437833])

Gözlemleri eğitim ve doğrulama setlerine bu şekilde böldüğümüzde, **doğrulama seti hata oranları** sırasıyla **lineer model için 20,76**, **kuadratik model için 16,95** ve **kübik model için 16,97** olarak bulunmuştur.

Bu sonuçlar önceki bulgularımızla tutarlıdır: **Horsepower’ın kuadratik fonksiyonunu kullanan bir model**, sadece lineer fonksiyon kullanan bir modele göre daha iyi performans göstermektedir ve **kübik fonksiyon kullanmanın ek bir iyileştirme sağladığına dair bir kanıt yoktur.**

# 5.3.2 Cross-Validation

Kuramsal olarak, **çapraz doğrulama tahmini** herhangi bir genelleştirilmiş doğrusal model (GLM) için hesaplanabilir. Ancak uygulamada, Python’da çapraz doğrulamanın en basit yolu **sklearn** kullanmaktır; sklearn’in arayüzü (API) **statsmodels**’dan farklıdır. Biz bugüne kadar GLM’leri **statsmodels** ile uydurduk.

Bu, veri bilimcilerin sıkça karşılaştığı bir problemdir:

> “A görevini yapan bir fonksiyonum var ve bunu B görevini gerçekleştiren bir şeye vermem gerekiyor, böylece B(A(D)) hesaplayabileyim, burada D veri setim.”

A ve B birbirleriyle doğrudan uyumlu olmadığında, bir **wrapper** kullanmak gerekir. **ISLP wrapper paketi**, **sklearn_sm()** adlı bir wrapper sağlar; bu sayede **statsmodels ile uydurulmuş modelleri**, **sklearn’in çapraz doğrulama araçlarıyla** kolayca kullanabiliriz.

`sklearn_sm()` sınıfının ilk argümanı bir **statsmodels modeli**dir.
İki ek opsiyonel argüman alabilir:

* **model_str:** Bir formül belirtmek için kullanılır.
* **model_args:** Modeli uydururken kullanılacak ek argümanların bulunduğu bir sözlük (dictionary). Örneğin, lojistik regresyon modeli için **family** argümanını belirtmek gerekir:

  ```python
  model_args = {'family': sm.families.Binomial()}
  ```

İşte wrapper’ımızın kullanımına bir örnek:


In [9]:
hp_model = sklearn_sm(sm.OLS, MS(['horsepower']))
X, Y = Auto.drop(columns=['mpg']), Auto['mpg']
cv_results = cross_validate(hp_model,
                            X,
                            Y,
                            cv=Auto.shape[0])
cv_err = np.mean(cv_results['test_score'])
cv_err

24.231513517929212

`cross_validate()` fonksiyonuna verilen argümanlar şunlardır:

* `fit()`, `predict()` ve `score()` metodlarına sahip bir nesne,
* Özellikler matrisi **X**,
* Hedef değişken **Y**.

Ayrıca `cross_validate()`’a **cv** adında ek bir argüman da verdik; **K** tam sayısını belirtmek, **K-katlı çapraz doğrulama** (K-fold cross-validation) yapılmasını sağlar. Biz, toplam gözlem sayısına karşılık gelen bir değer verdik; bu da **tek bırakma çapraz doğrulama (LOOCV)** anlamına gelir.

`cross_validate()` fonksiyonu bir sözlük (dictionary) üretir; biz burada yalnızca **çapraz doğrulamalı test skoru (MSE)** ile ilgileniyoruz, bu da **24,23** olarak tahmin edilmiştir.

Bu işlemi, giderek daha karmaşık **polinom uyumlar** için tekrarlayabiliriz. Bunu otomatikleştirmek için bir **for döngüsü** kullanıyoruz; döngü, **1’den 5’e kadar dereceli polinom regresyonlarını** uydurur, ilişkili çapraz doğrulama hatasını hesaplar ve bunu `cv_error` vektörünün **i’inci elemanına** kaydeder.

For döngüsündeki **d değişkeni**, polinomun derecesini temsil eder. Öncelikle, **vektörü başlatıyoruz**; bu komut birkaç saniye sürebilir.


In [10]:
cv_error = np.zeros(5)  # 1'den 5'e kadar dereceler için çapraz doğrulama hatalarını saklayacak vektörü başlatıyoruz

H = np.array(Auto['horsepower'])  # 'horsepower' sütununu bir NumPy dizisine çeviriyoruz
M = sklearn_sm(sm.OLS)            # statsmodels OLS modelini sklearn uyumlu hale getiren wrapper'ı oluşturuyoruz

for i, d in enumerate(range(1,6)):
    X = np.power.outer(H, np.arange(d+1))  # H sütunundan 0'dan d derecesine kadar polinom özellik matrisi oluşturuyoruz
    M_CV = cross_validate(M,
                          X,
                          Y,
                          cv=Auto.shape[0])  # LOOCV kullanarak çapraz doğrulama yapıyoruz
    cv_error[i] = np.mean(M_CV['test_score'])  # Test skorlarının ortalamasını alıp cv_error dizisine kaydediyoruz

cv_error  # 1'den 5'e kadar polinom derecelerinin çapraz doğrulama hatalarını gösterir

array([24.23151352, 19.24821312, 19.33498406, 19.42443031, 19.03320428])

Şekil 5.4’te olduğu gibi, **tahmin edilen test MSE’sinde** lineer ve kuadratik uyumlar arasında belirgin bir düşüş görülmektedir; ancak **daha yüksek dereceli polinomlar kullanıldığında** belirgin bir iyileşme yoktur.

Yukarıda **`np.power().outer()`** yöntemini tanıttık.

* `outer()` metodu, **iki argüman alan bir işleme** uygulanır (örneğin `add()`, `min()`, `power()` gibi).
* İki dizi (array) alır ve daha büyük bir dizi oluşturur; burada işlem, iki dizinin **her bir eleman çifti** için uygulanır.

Özetle, `np.power.outer(H, np.arange(d+1))` ifadesi, her bir gözlem için polinom derecelerini hesaplamak amacıyla bu mantığı kullanır.

In [11]:
 A = np.array([3, 5, 9])
 B = np.array([2, 4])
 np.add.outer(A, B)

array([[ 5,  7],
       [ 7,  9],
       [11, 13]])

Yukarıdaki çapraz doğrulama (CV) örneğinde **K = n** kullanmıştık; ancak elbette **K < n** de kullanılabilir. Kod yukarıdakine çok benzer ve **daha hızlı çalışır**.

Burada, veriyi **K = 10 rastgele gruba** bölmek için `KFold()` kullanıyoruz.

* `random_state` ile bir **rastgelelik tohumu** belirliyoruz.
* Ayrıca, 1’den 5’e kadar dereceli polinom uyumlarının karşılık gelen **CV hatalarını saklayacağımız `cv_error` vektörünü** başlatıyoruz.


In [12]:
cv_error = np.zeros(5)
cv = KFold(n_splits=10,
            shuffle=True,
            random_state=0) # use same splits for each degree
for i, d in enumerate(range(1,6)):
    X = np.power.outer(H, np.arange(d+1))
    M_CV = cross_validate(M,
    X,
    Y,
    cv=cv)
    cv_error[i] = np.mean(M_CV['test_score'])
cv_error

array([24.20766449, 19.18533142, 19.27626666, 19.47848404, 19.13722016])

Dikkat edilirse, **hesaplama süresi LOOCV’ye göre çok daha kısadır.**
(Teoride, en küçük kareler doğrusal model için LOOCV hesaplama süresi, LOOCV formülü (5.2) mevcut olduğu için K-katlı CV’ye göre daha hızlı olmalıdır; ancak `cross_validate()` fonksiyonu bu formülü kullanmaz.)

Yine de, **kübik veya daha yüksek dereceli polinom terimleri kullanmanın**, sadece kuadratik bir uyum kullanmaya kıyasla test hatasını düşürdüğüne dair çok az kanıt görülmektedir.

`cross_validate()` fonksiyonu esnektir ve farklı **veri bölme mekanizmalarını** argüman olarak alabilir.
Örneğin, **`ShuffleSplit()`** fonksiyonu kullanılarak doğrulama seti yöntemi, K-katlı çapraz doğrulama kadar kolay bir şekilde uygulanabilir.


In [13]:
validation = ShuffleSplit(n_splits=1,
                            test_size=196,
                            random_state=0)
results = cross_validate(hp_model,
                            Auto.drop(['mpg'], axis=1),
                            Auto['mpg'],
                            cv=validation);
results['test_score']

array([23.61661707])

Test hatasındaki **değişkenliği tahmin etmek** için aşağıdaki kod çalıştırılabilir:


In [14]:
validation = ShuffleSplit(n_splits=10,
                            test_size=196,
                            random_state=0)
results = cross_validate(hp_model,
                            Auto.drop(['mpg'], axis=1),
                            Auto['mpg'],
                            cv=validation)
results['test_score'].mean(), results['test_score'].std()

(23.802232661034164, 1.4218450941091847)

Dikkat edilmelidir ki, bu **standart sapma**, ortalama test skoru veya bireysel skorların **örnekleme değişkenliğinin geçerli bir tahmini** değildir; çünkü rastgele seçilen eğitim örnekleri **çakışır** ve bu da **korelasyonlar** yaratır. Yine de, farklı rastgele katları seçmenin neden olduğu **Monte Carlo değişkenliği** hakkında bir fikir verir.

# 5.3.3 Bootstrap

Bootstrap yöntemini, **Bölüm 5.2’deki basit örnek** üzerinde ve ayrıca **Auto veri setinde doğrusal regresyon modelinin doğruluğunu tahmin etme** örneğinde gösteriyoruz.

## Bir İlgili İstatistiğin Doğruluğunu Tahmin Etmek

Bootstrap yaklaşımının büyük avantajlarından biri, **neredeyse her durumda uygulanabilir** olmasıdır. Karmaşık matematiksel hesaplamalar gerekmez. Python’da bootstrap için çeşitli uygulamalar olsa da, **standart hatayı tahmin etmek** için kullanımı yeterince basittir; bu yüzden verilerimiz bir dataframe’de saklandığında kullanabileceğimiz kendi fonksiyonumuzu aşağıda yazıyoruz.

Bootstrap’i göstermek için basit bir örnekle başlıyoruz. **ISLP paketindeki Portfolio veri seti**, Bölüm 5.2’de açıklanmıştır. Amaç, formül (5.7) ile verilen **α parametresinin örnekleme varyansını** tahmin etmektir.

Bunun için `alpha_func()` adında bir fonksiyon oluşturacağız:

* Girdi olarak, **X ve Y sütunlarını içeren bir dataframe D** ve
* Hangi gözlemlerin α’yı tahmin etmek için kullanılacağını belirten bir **idx vektörü** alır.
* Fonksiyon, seçilen gözlemler temelinde **α tahminini** çıktılar.

In [15]:
Portfolio = load_data('Portfolio')

def alpha_func(D, idx):
    cov_ = np.cov(D[['X','Y']].loc[idx], rowvar=False)
    return ((cov_[1,1]- cov_[0,1]) / (cov_[0,0]+cov_[1,1]-2*cov_[0,1]))

Bu fonksiyon, **idx argümanı ile belirtilen gözlemlere** formül (5.7)deki **minimum varyans formülünü** uygulayarak **α tahminini** döndürür.

Örneğin, aşağıdaki komut **tüm 100 gözlem kullanılarak α’yı** tahmin eder.


In [16]:
alpha_func(Portfolio, range(100))

0.57583207459283

Sonraki adımda, **0–99 aralığından 100 gözlemi rastgele ve tekrarlı seçimle** seçiyoruz.
Bu, **yeni bir bootstrap veri seti oluşturmak** ve **ˆα’yı yeni veri setine göre yeniden hesaplamak** ile eşdeğerdir.


In [17]:
rng = np.random.default_rng(0)
alpha_func(Portfolio,
            rng.choice(100,
            100,
            replace=True))

0.6074452469619004

Bu süreç, yalnızca bir **dataframe** argümanı alan **rastgele fonksiyonlar** için **bootstrap standart hatasını** hesaplayan basit bir `boot_SE()` fonksiyonu oluşturacak şekilde genellenebilir.


In [18]:
def boot_SE(func,
            D,
            n=None,
            B=1000,
            seed=0):
    rng = np.random.default_rng(seed)
    first_, second_ = 0, 0
    n = n or D.shape[0]
    for _ in range(B):
        idx = rng.choice(D.index,
                            n,
                            replace=True)
        value = func(D, idx)
        first_ += value
        second_ += value**2
    return np.sqrt(second_ / B- (first_ / B)**2)

Dikkat edin, `for _ in range(B)` ifadesinde **_** döngü değişkeni olarak kullanılmıştır. Bu, sayacın değerinin önemli olmadığı durumlarda sıkça kullanılır ve döngünün **B kez çalıştırılmasını** sağlar.

Şimdi, **B = 1.000 bootstrap tekrarı** kullanarak α tahminimizin doğruluğunu değerlendirmek için fonksiyonumuzu kullanalım.


In [19]:
alpha_SE = boot_SE(alpha_func,
                    Portfolio,
                    B=1000,
                    seed=0)
alpha_SE

0.09118176521277699

Sonuç olarak, **bootstrap ile SE(ˆα) tahmini 0,0912** olarak bulunmuştur.

## Doğrusal Regresyon Modelinin Doğruluğunu Tahmin Etmek

Bootstrap yöntemi, bir istatistiksel öğrenme yönteminden elde edilen **katsayı tahminlerinin ve öngörülerin değişkenliğini** değerlendirmek için kullanılabilir.

Burada, **Auto veri setinde `horsepower` kullanarak `mpg` tahmin eden doğrusal regresyon modeli** için:

* **β₀** (intercept) ve **β₁** (slope) tahminlerinin değişkenliğini değerlendirmek amacıyla bootstrap yaklaşımını kullanıyoruz.
* Bootstrap ile elde edilen tahminleri, Bölüm 3.1.2’de açıklanan **SE(ˆβ₀) ve SE(ˆβ₁) formülleri** ile karşılaştıracağız.

`boot_SE()` fonksiyonumuzu kullanabilmek için, ilk argümanı olarak **bir dataframe D ve indeksler idx alan bir fonksiyon** yazmamız gerekir. Ancak burada, **belirli bir regresyon modeli** (formül ve veri ile belirtilmiş) için bootstrap yapacağız.

Bunu yapmak için birkaç basit adım vardır:

1. `boot_OLS()` adında **genel bir bootstrap fonksiyonu** yazıyoruz; bu fonksiyon, regresyonu tanımlayan bir **formül** alır.
2. `clone()` fonksiyonunu kullanarak formülün bir kopyasını oluşturuyoruz; böylece bu formül **yeniden örneklenmiş veri setine** uygulanabilir.
3. Bu sayede, `poly()` gibi türetilmiş özellikler de **yeniden örneklenen veri seti üzerinde yeniden uydurulur**.

In [25]:
def boot_OLS(model_matrix, response, D, idx):
    D_ = D.loc[idx]
    Y_ = D_[response]
    X_ = clone(model_matrix).fit_transform(D_)
    return sm.OLS(Y_, X_).fit().params

Bu, `boot_SE()`’in ilk argümanı olarak tam olarak gereken şey değildir.

Bootstrap sürecinde değişmeyecek **modeli belirten ilk iki argüman** vardır ve biz bunları **sabitlemek** isteriz. Bunun için **`functools` modülündeki `partial()`** fonksiyonu kullanılır:

* `partial()` bir fonksiyonu argüman olarak alır ve **bazı argümanları soldan başlayarak sabitler**.
* Biz de bunu kullanarak, `boot_OLS()` fonksiyonunun **ilk iki model-formül argümanını sabitliyoruz.**


In [21]:
hp_func = partial(boot_OLS, MS(['horsepower']), 'mpg')

`hp_func?` yazarak fonksiyonun **D ve idx olmak üzere iki argümanı** olduğunu görebiliriz — bu, **ilk iki argümanı sabitlenmiş bir `boot_OLS()` versiyonudur** ve bu nedenle `boot_SE()` için ideal bir ilk argümandır.

Artık `hp_func()` fonksiyonu, gözlemler arasından **tekrarlı rastgele örnekleme** yaparak **intercept ve slope terimleri için bootstrap tahminleri** oluşturmak amacıyla kullanılabilir. Öncelikle, fonksiyonun işlevini **10 bootstrap örneği** üzerinde gösteriyoruz.


In [26]:
rng = np.random.default_rng(0)  # Rastgele sayı üreteci için bir tohum (seed) belirliyoruz
np.array([hp_func(Auto.reset_index(),        # hp_func fonksiyonunu kullanıyoruz
                  rng.choice(392,  # 392 gözlem arasından
                  392,            # 392 gözlem seçiyoruz
                  replace=True))  # Tekrarlı seçimle (bootstrap)
           for _ in range(10)]) # 10 bootstrap örneği oluşturuyoruz

array([[39.88064456, -0.1567849 ],
       [38.73298691, -0.14699495],
       [38.31734657, -0.14442683],
       [39.91446826, -0.15782234],
       [39.43349349, -0.15072702],
       [40.36629857, -0.15912217],
       [39.62334517, -0.15449117],
       [39.0580588 , -0.14952908],
       [38.66688437, -0.14521037],
       [39.64280792, -0.15555698]])

Sonra, intercept ve slope terimleri için 1.000 bootstrap tahmini kullanarak standart hataları hesaplamak için **boot_SE()** fonksiyonunu kullanıyoruz.


In [27]:
hp_se = boot_SE(hp_func,
                Auto,
                B=1000,
                seed=10)
hp_se

intercept     0.731176
horsepower    0.006092
dtype: float64

Bu, SE(ˆβ0) için bootstrap tahmininin 0,85 ve SE(ˆβ1) için bootstrap tahmininin 0,0074 olduğunu gösterir. Bölüm 3.1.2’de tartışıldığı gibi, doğrusal bir modelde regresyon katsayılarının standart hatalarını hesaplamak için standart formüller kullanılabilir. Bunlar, ISLP.sm içindeki **summarize()** fonksiyonu kullanılarak elde edilebilir.


In [28]:
hp_model.fit(Auto, Auto['mpg'])
model_se = summarize(hp_model.results_)['std err']
model_se

intercept     0.717
horsepower    0.006
Name: std err, dtype: float64

ˆβ0 ve ˆβ1 için Bölüm 3.1.2’deki formüller kullanılarak elde edilen standart hata tahminleri, intercept için 0,717 ve slope için 0,006’dır. İlginç bir şekilde, bunlar bootstrap ile elde edilen tahminlerden biraz farklıdır. Peki bu bootstrap ile ilgili bir sorun olduğunu mu gösterir? Aslında, tam tersi bir durumu işaret eder.

Hatırlarsak, sayfa 75’teki Denklem 3.8’de verilen standart formüller belirli varsayımlara dayanır. Örneğin, bilinmeyen parametre σ²’ye, yani gürültü varyansına bağlıdırlar. σ² daha sonra RSS kullanılarak tahmin edilir. Standart hata formülleri doğrusal modelin doğru olmasına dayanmasa da, σ²’nin tahmini buna dayanır. Sayfa 99’daki Şekil 3.8’de veride doğrusal olmayan bir ilişki olduğu görülmektedir; bu nedenle doğrusal bir uyumdan elde edilen artıklar şişirilmiş olur ve ˆσ² de büyük olur.

İkinci olarak, standart formüller (biraz gerçekçi olmayan bir şekilde) xi’nin sabit olduğunu ve tüm değişkenliğin hatalardan (ϵi) geldiğini varsayar. Bootstrap yaklaşımı ise bu varsayımlara dayanmaz ve bu nedenle ˆβ0 ve ˆβ1’in standart hatalarını sm.OLS’den elde edilen sonuçlara kıyasla daha doğru tahmin ediyor olabilir.

Aşağıda, bootstrap standart hata tahminlerini ve veriye ikinci dereceden (quadratic) model uygulanarak elde edilen standart lineer regresyon tahminlerini hesaplıyoruz. Bu model veriye iyi bir uyum sağladığı için (Şekil 3.8), artık bootstrap tahminleri ile SE(ˆβ0), SE(ˆβ1) ve SE(ˆβ2) için standart tahminler arasında daha iyi bir uyum vardır.


In [29]:
quad_model = MS([poly('horsepower', 2, raw=True)])
quad_func = partial(boot_OLS,
                    quad_model,
                    'mpg')
boot_SE(quad_func, Auto, B=1000)

intercept                                  1.538641
poly(horsepower, degree=2, raw=True)[0]    0.024696
poly(horsepower, degree=2, raw=True)[1]    0.000090
dtype: float64

Sonuçları, **sm.OLS()** kullanılarak hesaplanan standart hatalarla karşılaştırıyoruz.


In [30]:
M = sm.OLS(Auto['mpg'],
             quad_model.fit_transform(Auto))
summarize(M.fit())['std err']

intercept                                  1.800
poly(horsepower, degree=2, raw=True)[0]    0.031
poly(horsepower, degree=2, raw=True)[1]    0.000
Name: std err, dtype: float64